In [1]:
import sys
import os
from pathlib import Path

current_dir = str(Path(os.getcwd()).parent.parent)
print(f"Current dir: {current_dir}")
sys.path.insert(0, current_dir)

Current dir: d:\Programming\Python\comfyui-image-scorer


In [2]:
import numpy as np
from step03training.full_data.data_utils import (
    load_training_data,
    filter_unused_features,
    add_interaction_features,
    load_training_scores,
)

from shared.paths import vectors_file, scores_file, filtered_data

print("--- Step 1: Loading Data ---")
if not os.path.exists(filtered_data):
    x, y = load_training_data(vectors_file, scores_file)
else:
    x = np.array([])
    y = load_training_scores(scores_file)


print(f"Loaded Data Shape: {x.shape}")

print("\n--- Step 2: Filtering Unused Features ---")
# Removes features with zero variance or zero importance in a quick probe
x, kept_indices = filter_unused_features(x, y)
print(f"Filtered Data Shape: {x.shape}")

print("\n--- Step 3: Generating Interaction Features ---")
# Adds top 500 polynomial interactions (feature_A * feature_B)
x, _ = add_interaction_features(x, y, target_k=200)
print(f"Final Data Shape (with Interactions): {x.shape}")
print("Data Preparation Complete.")

prepare_dir: D:\Programming\Python\comfyui-image-scorer\step02prepare
--- Step 1: Loading Data ---
Loaded Data Shape: (11340, 2347)

--- Step 2: Filtering Unused Features ---
Filtering features... Initial shape: (11340, 2347)


Training LightGBM: 100%|█████████▉| 299/300 [00:24<00:00, 13.02it/s]

Dropped 262 unused features. New shape: (11340, 2085)


Training LightGBM: 100%|██████████| 300/300 [00:29<00:00, 10.33it/s]


Saved filtered data to cache: D:\Programming\Python\comfyui-image-scorer\step03training\output\filtered_data.npz
Filtered Data Shape: (11340, 2085)

--- Step 3: Generating Interaction Features ---
Scanning 2172570 potential interactions in batches of 246...


Computing Correlations: 100%|██████████| 11340/11340 [03:41<00:00, 51.29samples/s]


Selecting top 200 interaction features...


Building Interaction Matrix: 100%|██████████| 11340/11340 [00:56<00:00, 202.34samples/s]


Saved interaction data to cache: D:\Programming\Python\comfyui-image-scorer\step03training\output\interaction_data.npz
Final Data Shape (with Interactions): (11340, 2285)
Data Preparation Complete.


In [3]:
import random
from IPython.display import clear_output
from shared.config import config
from step03training.full_data.run import optimize_hyperparameters, evaluate_hyperparameter_combo
from step03training.full_data.config_utils import (
    crossover_config,
    generate_random_config,
    #generate_slowest_setup
)
from shared.paths import training_output_dir

# Setup Loop Variables
current_cfg = generate_random_config()
top_cfg = config["training"]["top"]
fastest_cfg = config["training"]["fastest"]
slowest_cfg = config["training"]["slowest"]
#slowest_cfg=generate_slowest_setup()
#config["training"]["slowest"]= slowest_cfg
max_iters = config["training"]["max_iters"]
max_combos = config["training"]["max_combos"]
print(f"Initialized HPO loop with max_iters={max_iters}, max_combos={max_combos}")

temp_model_base = os.path.join(training_output_dir, "temp_model")

get_base_score=True
if get_base_score:
    for value in ["top","fastest"]:
        print(f"getting {value} base score ...")
        current=config["training"][value]
        score, t_time = evaluate_hyperparameter_combo(current , temp_model_base, X=x, y=y)
        new_current = {**current , "best_score": score, "training_time": t_time}
        current.update(new_current)
        print (f"score: {score}")



for i in range(max_iters):
    if i%10==0 and i>0:
        clear_output(wait=True)
    # Refresh references
    top_cfg = config["training"]["top"]
    fastest_cfg = config["training"]["fastest"]
    slowest_cfg = config["training"]["slowest"]

    max_combos = config["training"]["max_combos"]

    # Strategy selection
    rand_val = random.random()
    if rand_val < 0.01:
        base_cfg = current_cfg
        strategy = "RANDOM_START"
    elif rand_val < 0.30:
        base_cfg = fastest_cfg
        strategy = "FASTEST"
    elif rand_val < 0.9:
        base_cfg = top_cfg
        strategy = "TOP"
    #elif rand_val < 0.80:
    else:
        # Crossover
        candidates = [
            c for c in [top_cfg, fastest_cfg, slowest_cfg] if c["best_score"] > -9999
        ]
        if len(candidates) < 2:
            candidates = [c for c in [top_cfg, fastest_cfg, current_cfg] if c]
            if len(candidates) < 2:
                candidates = [generate_random_config(), generate_random_config()]

        parents = random.sample(candidates, 2)
        base_cfg = crossover_config(dict(parents[0]), dict(parents[1]))
        strategy = "CROSSOVER"
    # else:
    #     base_cfg = slowest_cfg
    #     strategy = "SLOWEST"

    print(f"\nIter {i + 1}/{max_iters} — Strategy: {strategy}")
    print (f"current score; {base_cfg["best_score"]}")
    # optimize_hyperparameters handles updates and saving internally now.
    results = optimize_hyperparameters(
        base_cfg=base_cfg, max_combos=max_combos, X=x, y=y, strategy=strategy
    )

    # Info only loop
    for candidate_cfg, metrics in results:
        r2 = metrics["r2"]
        t_time = metrics["training_time"]

        # Check if this result matches current bests (just for log/info)
        is_top = r2 == top_cfg["best_score"]
        is_fastest = t_time == fastest_cfg["training_time"]

        if is_top:
            print(f"  [Info] This batch produced the current TOP score: {r2:.6f}")
        if is_fastest:
            print(
                f"  [Info] This batch produced the current FASTEST time: {t_time:.4f}s"
            )

    # Refresh local current_cfg for next RANDOM_START or Crossover inheritance
    if results:
        # Simply take the last one as 'current' for random drift
        current_cfg = results[-1][0]

print(f"\nFinished optimization.")
print(f"Top R2: {top_cfg['best_score']:.6f}")
print(
    f"Fastest Time: {fastest_cfg['training_time']:.6f}s (R2: {fastest_cfg['best_score']:.6f})"
)
print(
    f"Slowest (High Score) Time: {slowest_cfg['training_time']:.6f}s (R2: {slowest_cfg['best_score']:.6f})"
)


Iter 11/100 — Strategy: FASTEST
current score; 0.3843762492948568
Optimizing hyperparameters over grid: {'min_child_samples': [66, 60, 54], 'early_stopping_rounds': [170, 155, 139], 'subsample': [0.26933659958143147], 'learning_rate': [0.1522450537877656], 'max_depth': [2], 'reg_alpha': [6.2832836417984], 'num_leaves': [828], 'min_split_gain': [0.16356534921175353], 'n_estimators': [2000], 'colsample_bytree': [0.13374806020408], 'reg_lambda': [9.9]}
______________________________
Evaluating hyperparameter combo 1/9, with params: {'min_child_samples': 66, 'early_stopping_rounds': 170, 'subsample': 0.26933659958143147, 'learning_rate': 0.1522450537877656, 'max_depth': 2, 'reg_alpha': 6.2832836417984, 'num_leaves': 828, 'min_split_gain': 0.16356534921175353, 'n_estimators': 2000, 'colsample_bytree': 0.13374806020408, 'reg_lambda': 9.9}


Training LightGBM:  59%|█████▊    | 1174/2000 [00:05<00:03, 228.42it/s]


r2=0.3894, time=5.1400s for Evaluated params {'min_child_samples': 66, 'early_stopping_rounds': 170, 'subsample': 0.26933659958143147, 'learning_rate': 0.1522450537877656, 'max_depth': 2, 'reg_alpha': 6.2832836417984, 'num_leaves': 828, 'min_split_gain': 0.16356534921175353, 'n_estimators': 2000, 'colsample_bytree': 0.13374806020408, 'reg_lambda': 9.9}
--> Found new FASTEST model! Old: 0.3976 (Score: 0.3894, Time: 5.1400s)
______________________________
Evaluating hyperparameter combo 2/9, with params: {'min_child_samples': 66, 'early_stopping_rounds': 155, 'subsample': 0.26933659958143147, 'learning_rate': 0.1522450537877656, 'max_depth': 2, 'reg_alpha': 6.2832836417984, 'num_leaves': 828, 'min_split_gain': 0.16356534921175353, 'n_estimators': 2000, 'colsample_bytree': 0.13374806020408, 'reg_lambda': 9.9}


Training LightGBM:  58%|█████▊    | 1159/2000 [00:04<00:03, 232.80it/s]


r2=0.3894, time=4.9787s for Evaluated params {'min_child_samples': 66, 'early_stopping_rounds': 155, 'subsample': 0.26933659958143147, 'learning_rate': 0.1522450537877656, 'max_depth': 2, 'reg_alpha': 6.2832836417984, 'num_leaves': 828, 'min_split_gain': 0.16356534921175353, 'n_estimators': 2000, 'colsample_bytree': 0.13374806020408, 'reg_lambda': 9.9}
______________________________
Evaluating hyperparameter combo 3/9, with params: {'min_child_samples': 66, 'early_stopping_rounds': 139, 'subsample': 0.26933659958143147, 'learning_rate': 0.1522450537877656, 'max_depth': 2, 'reg_alpha': 6.2832836417984, 'num_leaves': 828, 'min_split_gain': 0.16356534921175353, 'n_estimators': 2000, 'colsample_bytree': 0.13374806020408, 'reg_lambda': 9.9}


...

r2=0.3976, time=5.5358s for Evaluated params {'reg_alpha': 6.2832836417984, 'subsample': 0.3584870140428854, 'min_split_gain': 0.1457367261476724, 'reg_lambda': 9.9, 'learning_rate': 0.16916117087529509, 'num_leaves': 819, 'early_stopping_rounds': 155, 'n_estimators': 1980, 'min_child_samples': 54, 'max_depth': 2, 'colsample_bytree': 0.147122866224488}
______________________________
Evaluating hyperparameter combo 5/9, with params: {'reg_alpha': 6.2832836417984, 'subsample': 0.32589728549353214, 'min_split_gain': 0.1457367261476724, 'reg_lambda': 9.9, 'learning_rate': 0.16916117087529509, 'num_leaves': 819, 'early_stopping_rounds': 155, 'n_estimators': 1980, 'min_child_samples': 54, 'max_depth': 2, 'colsample_bytree': 0.147122866224488}


Training LightGBM:  66%|██████▌   | 1302/1980 [00:05<00:02, 232.82it/s]


r2=0.3976, time=5.5924s for Evaluated params {'reg_alpha': 6.2832836417984, 'subsample': 0.32589728549353214, 'min_split_gain': 0.1457367261476724, 'reg_lambda': 9.9, 'learning_rate': 0.16916117087529509, 'num_leaves': 819, 'early_stopping_rounds': 155, 'n_estimators': 1980, 'min_child_samples': 54, 'max_depth': 2, 'colsample_bytree': 0.147122866224488}
______________________________
Evaluating hyperparameter combo 6/9, with params: {'reg_alpha': 6.2832836417984, 'subsample': 0.2933075569441789, 'min_split_gain': 0.1457367261476724, 'reg_lambda': 9.9, 'learning_rate': 0.16916117087529509, 'num_leaves': 819, 'early_stopping_rounds': 155, 'n_estimators': 1980, 'min_child_samples': 54, 'max_depth': 2, 'colsample_bytree': 0.147122866224488}


Training LightGBM:   0%|          | 0/1980 [00:00<?, ?it/s]

KeyboardInterrupt: 